In [1]:
import warnings
warnings.filterwarnings("ignore")

from fastai.vision import *
from fastai.metrics import error_rate
from fastai.callbacks import *
from fastai.vision.models.efficientnet import *
import torch

torch.cuda.empty_cache()

In [2]:
#masukkin image dataset nya disini, tapi per folder harus ada labelnya
#Reference Implementation fastai --->>> https://gilberttanner.com/blog/fastai-image-classification

defaults.device = torch.device("cuda")


In [3]:
def get_data(bs, size): 
    tfms = get_transforms(max_lighting=0.4, max_zoom=1.2, max_warp=0.2, max_rotate=20, xtra_tfms=[flip_lr()])
    return ImageDataBunch.from_folder(Path('./dataset'),
                                  train = 'train/',
                                  valid_pct = 0.1,
                                  resize_method=ResizeMethod.SQUISH, 
                                  ds_tfms = tfms,
                                  size = size,
                                  bs = bs,
                                  num_workers = 50
                                  ).normalize(imagenet_stats)

In [4]:
data = get_data(64, 128)

len(data.classes), data.c

(42, 42)

In [5]:
model = EfficientNet.from_pretrained('efficientnet-b5')
# model

Loaded pretrained weights for efficientnet-b5


In [6]:
model._fc = nn.Linear(model._fc.in_features, data.c)

In [7]:
from efficientnet_pytorch import EfficientNet

def get_model(pretrained=True, **kwargs): 
    model = EfficientNet.from_pretrained('efficientnet-b5')
    model._fc = nn.Linear(model._fc.in_features, data.c) # check the top most layer
    return model 

def stage1(learn, data=None):
    if data: 
        learn.data=data
        learn.to_fp16()
        
    learn.freeze()
    learn.fit_one_cycle(5)
    
def stage2(learn, load_filename=None): 
    learn.unfreeze()
    learn.lr_find()
    learn.recorder.plot(suggestion=True)
    min_grad_lr = learn.recorder.min_grad_lr
    learn.fit_one_cycle(5, slice(min_grad_lr/40, min_grad_lr))

learn = Learner(data, model,
               metrics = [accuracy], 
                bn_wd=False, #disable weight decay 
                loss_func = LabelSmoothingCrossEntropy(), 
                callback_fns=[BnFreeze,
                             partial(SaveModelCallback, monitor='accuracy', name='most_accurate')],
               path='./dataset').to_fp16() # because different clf layer, we use Learner.
learn.split(lambda m: (model._conv_head,))

Learner(data=ImageDataBunch;

Train: LabelList (94853 items)
x: ImageList
Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128)
y: CategoryList
00,00,00,00,00
Path: dataset;

Valid: LabelList (10539 items)
x: ImageList
Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128)
y: CategoryList
31,18,39,13,09
Path: dataset;

Test: None, model=EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0

In [ ]:
stage1(learn)


epoch,train_loss,valid_loss,accuracy,time
0,2.597157,2.467558,0.441313,10:46


Better model found at epoch 0 with accuracy value: 0.4413132071495056.


In [ ]:
stage2(learn)

In [ ]:
learn.save('b5-epoch5-128')
learn.load('b5-epoch5-128', purge=True)



In [ ]:
data = get_data(64, 256)
stage1(learn, data)

In [ ]:
stage2(learn)

In [ ]:
learn.save('b5-epoch5-256')
learn.load('b5-epoch5-256', purge=True)


In [ ]:
data = get_data(16, 384)
stage1(learn, data)

In [ ]:
stage2(learn)

In [ ]:
learn.save('b5-epoch5-256')
learn.load('b5-epoch5-256', purge=True)


In [ ]:
def make_submission(learn, filename):
  log_preds, test_labels = learn.get_preds(ds_type=DatasetType.Test)
  preds = np.argmax(log_preds, 1)
  a = np.array(preds)
  submission = pd.DataFrame({'image_name': os.listdir('data/test'), 'label': a})
  submission.to_csv(path/filename, index=False)